

```
# Set Up
```



In [2]:
#Cloning the APS360Project repo
!git clone https://github.com/Marneus981/Aps360Project.git

fatal: destination path 'Aps360Project' already exists and is not an empty directory.


In [3]:
#Only run if new data has been uploaded since last git clone
#%cd /content/Aps360Project
#!git pull

In [4]:
#Setup


#Imports taken from tutorial example
import torch
import os
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt # plotting
import torch.optim as optim #gradient descent
import torchvision #to make things easier
from torchvision import models, datasets, transforms #migght be redundant
from torch.utils.data.sampler import SubsetRandomSampler# to use the data loader
from torch.utils.data import TensorDataset #load the tensordataset
from torch.utils.data import TensorDataset, DataLoader
from PIL import Image

#Ill be using hardware acceleration by means of a GPU 
use_cuda = True




In [5]:
#For reproducibility

seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `pytorch` pseudo-random generator at a fixed value
torch.manual_seed(seed_value)
torch.cuda.manual_seed(seed_value)

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
path = "/content/gdrive/.shortcut-targets-by-id/1fEMnaweRPt_MZzfehmDCWykeBCjFet9r/APS360 Project/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Data Loading

In [ ]:
#Data loading

# data_path = "/content/Aps360Project/Project_Data_Big"
data_path = path + "Project_Data_Sorted"
#extract images and labels from dataset folder and put into arrays
images = []
label = []
counter = 0
for root, dirs, files in os.walk(data_path):
  # if counter > 48:
    # break
  for file in files:
    try:
      filename = str(file)
      if (filename.endswith('.png') and not filename.startswith('._')):
        #print(str(root))
        filepath = str(root) + '/' + str(file)
        img = plt.imread(filepath, 0)
        img_cropped = img[0:154, 0:1200]
        images.append(img_cropped)
      if (filename.endswith('.semantic') and not filename.startswith('._')):
        filepath = str(root) + '/' + str(file)
        f = open(filepath, 'r')
        contents = f.read()
        elements = contents.split("\t", 50)
        #print(elements)
        semantic = []
        for element in elements:
          #print(element)
          if element.startswith('note'):
            element = element[5:8]
            if element.endswith('_'):
              element = element[:-1]
            semantic.append(element)

        label.append(semantic)
      counter+=1
    except:
      continue

#example image and label
plt.imshow(images[3]) 
print(label[3])

#convert labels to integer encoding

#helper functions

letters_to_numbers =  {
  'C':0,
  'D':2,
  'E':4,
  'F':5,
  'G':7,
  'A':9,
  'B':11
  }
accidentals_to_numbers = {
    'x':2,
    '#':1,
    '':0,
    'n':0,
    'b':-1,
    'bb':-2
}
def letter_to_int(noteId, noteDict):
  #Get octave
  octaveNum = int(noteId[-1])
  #Get letter name
  letterName = noteId[0]
  letterNum = letters_to_numbers[letterName]
  #Get accidental  
  accidentalName = noteId[1:-1]
  accidentalNum = accidentals_to_numbers[accidentalName]
  #Add together
  noteNum = (octaveNum+1)*12+letterNum+accidentalNum
  noteDict[noteNum] = noteId
  return noteNum

#call functions on label set to generate integer labels
int_label = []
noteDict = {}
for semantic in label:
  int_semantic = []
  for note in semantic:
    int_semantic.append(letter_to_int(note, noteDict))
  int_label.append(int_semantic)  



In [8]:
#Create list of tensors (to be converted to a tensor)
image_tensors_list = [torch.from_numpy(item).float() for item in images]
label = []
images = []

In [9]:
# torch.save(label, '/content/label_list.pt')

In [10]:
# label = []

In [11]:
#label = torch.load('/content/label_list.pt')
#print(label)

In [12]:
# torch.save(image_tensors_list, '/content/image_tensors_list.pt')

In [13]:
# images = []
#image_tensors_list = []

In [14]:
#image_tensors_list = torch.load('/content/image_tensors_list.pt')

In [15]:
#Code to figure out padding for data (images)
# for i in range(10):
#   print(image_tensors_list[i].shape)
# print(image_tensors_list[0].shape[0])
# print(image_tensors_list[0].shape[1])
# print(image_tensors_list[0].shape[2])

max_along0 = 0
max_along1 = 0
for i in range(len(image_tensors_list)):
  if ( image_tensors_list[i].shape[0] > max_along0 ):
    max_along0 = image_tensors_list[i].shape[0]
  if ( image_tensors_list[i].shape[1] > max_along1 ):
    max_along1 = image_tensors_list[i].shape[1]
print(max_along0)
print(max_along1)

#Find max lenght of labels for padding
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
  
    return maxList, maxLength 
maxList, maxLength  = FindMaxLength(int_label)

print(maxLength)

154
1200
47


In [16]:
# #Find max lenght of labels for padding
# def FindMaxLength(lst): 
#     maxList = max((x) for x in lst) 
#     maxLength = max(len(x) for x in lst ) 
  
#     return maxList, maxLength 
# maxList, maxLength  = FindMaxLength(int_label)

# print(maxLength)


In [17]:
#padding list of labels with blank notes (0s)

for i in range(len(int_label)):
  int_label[i] += [0] * (maxLength - len( int_label[i]))

#padding list of tensors (data) with a bunch or zeros
target = torch.zeros(30, 35, 512)
source = torch.ones(30, 35, 49)

for i in range(len(image_tensors_list)):
  target = torch.zeros(max_along0, max_along1, 3)
  target[:image_tensors_list[i].shape[0], :image_tensors_list[i].shape[1], :] = image_tensors_list[i]
  image_tensors_list[i] = target
#convert image into a tensor
#create tensor of data


image_tensors_list_tensor = torch.stack(image_tensors_list)
print(image_tensors_list_tensor.shape)
image_tensors_list_tensor = image_tensors_list_tensor.permute(0,3,1,2)

torch.Size([2048, 154, 1200, 3])


In [18]:
#image_tensors_list = []

In [19]:
#torch.save(image_tensors_list_tensor, '/content/image_tensors_tensor.pt')

In [20]:
# print(image_tensors_list_tensor.permute(0,3,1,2).size())

In [21]:
# image_tensors_list_tensor = image_tensors_list_tensor.permute(0,3,1,2)

In [1]:
#split labels
training_labels = int_label[:1638]
# validation_labels = int_label[1638:1843]
# testing_labels = int_label[1843:]

#transform tensors to labels

training_labels_tensor = torch.FloatTensor(training_labels)
# validation_labels_tensor = torch.FloatTensor(validation_labels)
# testing_labels_tensor = torch.FloatTensor(testing_labels)

#split data (180 samples not 320)
# training_data = images[:108]
# validation_data = images[108:144]
# testing_data = images[144:]

#split tensor data
training_tensor= image_tensors_list_tensor[:1638]
# validation_tensor = image_tensors_list_tensor[1638:1843]
# testing_tensor = image_tensors_list_tensor[1843:]

# save_loader(training_tensor, training_labels_tensor, 64, "training")
training_tensor = []
training_labels_tensor = []
# image_tensors_list = []

NameError: ignored

In [ ]:
print(training_labels_tensor.shape)

torch.Size([1638, 47])


In [ ]:
print(image_tensors_list[0].shape)
print(len(image_tensors_list))

torch.Size([154, 1200, 3])
2048


In [ ]:
print(len(int_label))
print(len(training_labels))
print(len(validation_labels))
print(len(testing_labels))

2048
1638
205
205


In [ ]:
# max(map(max, int_label))

90

In [ ]:
# image_tensors_list = []
#image_tensors_list_tensor = torch.Tensor(0)

In [ ]:
# print(image_tensors_list)
#print(image_tensors_list_tensor)

[]


Need to drop last because inconsistent batch sizes don't work on our CRNN

In [1]:
batch_size = 64
#"Final" datasets and dataloaders
# training_dataset = TensorDataset(training_tensor,training_labels_tensor) # training dataset
# training_loader = DataLoader(training_dataset,batch_size=batch_size, shuffle=True, drop_last=True) # create your dataloader
# validation_dataset = TensorDataset(validation_tensor,validation_labels_tensor) # training dataset
# validation_loader = DataLoader(validation_dataset,batch_size=batch_size, shuffle=True, drop_last=True) # create your dataloader
# testing_dataset = TensorDataset(testing_tensor,testing_labels_tensor) # training dataset
# testing_loader = DataLoader(testing_dataset,batch_size=batch_size, shuffle=True, drop_last=True) # create your dataloader


save_loader(validation_tensor, validation_labels_tensor, batch_size, "validation")
save_loader(testing_tensor, testing_labels_tensor, batch_size, "testing")

NameError: ignored

Need to drop last because inconsistent batch sizes don't work on our CRNN




##Data Loading old

In [8]:
# data_path = path + "Project_Data_Sorted"
data_path = "/content/Aps360Project/Project_Data_Big"
loader_path = path + "Dataloaders/"

In [9]:
def create_dataloaders(batch_size=64):
  image_tensors_list_tensor, int_label, stopped_root = create_tensor_list()
  
  #split labels
  training_labels = int_label[:1638]
  validation_labels = int_label[1638:1843]
  testing_labels = int_label[1843:]

  #transform tensors to labels

  training_labels_tensor = torch.FloatTensor(training_labels)
  validation_labels_tensor = torch.FloatTensor(validation_labels)
  testing_labels_tensor = torch.FloatTensor(testing_labels)

  #split data (180 samples not 320)
  # training_data = images[:108]
  # validation_data = images[108:144]
  # testing_data = images[144:]

  #split tensor data
  training_tensor= image_tensors_list_tensor[:1638]
  validation_tensor = image_tensors_list_tensor[1638:1843]
  testing_tensor = image_tensors_list_tensor[1843:]

  #"Final" datasets and dataloaders
  save_loader(training_tensor, training_labels_tensor, batch_size, "training")
  save_loader(validation_tensor, validation_labels_tensor, batch_size, "validation")
  save_loader(testing_tensor, testing_labels_tensor, batch_size, "testing")

  return stopped_root


In [10]:
#Data loading

#convert labels to integer encoding

#helper functions

letters_to_numbers =  {
  'C':0,
  'D':2,
  'E':4,
  'F':5,
  'G':7,
  'A':9,
  'B':11
  }
accidentals_to_numbers = {
    'x':2,
    '#':1,
    '':0,
    'n':0,
    'b':-1,
    'bb':-2
}

def letter_to_int(noteId, noteDict):
  #Get octave
  octaveNum = int(noteId[-1])
  #Get letter name
  letterName = noteId[0]
  letterNum = letters_to_numbers[letterName]
  #Get accidental  
  accidentalName = noteId[1:-1]
  accidentalNum = accidentals_to_numbers[accidentalName]
  #Add together
  noteNum = (octaveNum+1)*12+letterNum+accidentalNum
  noteDict[noteNum] = noteId
  return noteNum

def get_int_label(label):
  #call functions on label set to generate integer labels
  int_label = []
  noteDict = {}
  for note in label:
    # int_semantic = []
    # for note in semantic:
    int_label.append(letter_to_int(note, noteDict))
    # int_label.append(int_semantic)  
  return int_label

#Find max lenght of labels for padding
def FindMaxLength(lst): 
  maxList = max((x) for x in lst) 
  maxLength = max(len(x) for x in lst ) 

  return maxList, maxLength 


In [11]:
def create_tensor_list(start_at=None):
  image_tensors_list = []
  int_label = []
  counter = 0
  for root, dirs, files in os.walk(data_path):
    if start_at and root != start_at:
      continue
    if start_at and root == start_at:
      start_at = None
    if counter > 2048:
      start_at = root
      break
    for file in files:
      filename = str(file)
      try:
        if (filename.endswith('.png') and not filename.startswith('._')):
          #print(str(root))
          filepath = str(root) + '/' + str(file)
          img = plt.imread(filepath, 0)
          img_cropped = img[0:154, 0:1200]
          image_tensors_list.append(torch.from_numpy(img_cropped).float())
        if (filename.endswith('.semantic') and not filename.startswith('._')):
          filepath = str(root) + '/' + str(file)
          f = open(filepath, 'r')
          contents = f.read()
          elements = contents.split("\t", 50)
          #print(elements)
          semantic = []
          for element in elements:
            #print(element)
            if element.startswith('note'):
              element = element[5:8]
              if element.endswith('_'):
                element = element[:-1]
              semantic.append(element)      
          int_label.append(get_int_label(semantic))
        counter+=1
      except:
        continue
  print(counter)
  #Create list of tensors (to be converted to a tensor)
  max_along0 = 0
  max_along1 = 0
  for i in range(len(image_tensors_list)):
    if ( image_tensors_list[i].shape[0] > max_along0 ):
      max_along0 = image_tensors_list[i].shape[0]
    if ( image_tensors_list[i].shape[1] > max_along1 ):
      max_along1 = image_tensors_list[i].shape[1]
  print(max_along0)
  print(max_along1)
  #Find max lenght of labels for padding
  maxList, maxLength  = FindMaxLength(int_label)
  print(maxLength)


  #padding list of labels with blank notes (0s)
  for i in range(len(int_label)):
    int_label[i] += [0] * (maxLength - len( int_label[i]))

  #padding list of tensors (data) with a bunch or zeros
  target = torch.zeros(30, 35, 512)
  source = torch.ones(30, 35, 49)

  for i in range(len(image_tensors_list)):
    target = torch.zeros(max_along0, max_along1, 3)
    target[:image_tensors_list[i].shape[0], :image_tensors_list[i].shape[1], :] = image_tensors_list[i]
    image_tensors_list[i] = target
  #convert image into a tensor
  #create tensor of data

  image_tensors_list_tensor = torch.stack(image_tensors_list)
  print(image_tensors_list_tensor.shape)
  image_tensors_list_tensor = image_tensors_list_tensor.permute(0,3,1,2)

  return image_tensors_list_tensor, int_label, start_at



In [12]:
s = create_dataloaders()
print(s)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)


2050
154
1200
25


RuntimeError: ignored

In [2]:
def save_loader(tensor, labels, batch_size, name):
  filename = loader_path + "/" + name + "_" + str(batch_size) + '_loader.pth'
  mononame = loader_path + "/" + name + "_1_loader.pth"
  dataset = TensorDataset(tensor, labels)
  # loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
  mono_loader = DataLoader(dataset, batch_size=1)
  # torch.save(loader, filename)
  torch.save(mono_loader, mononame)